# global variables

In [1]:
# colab options
RUNNING_ON_COLAB = 'google.colab' in str(get_ipython())

# data options
INPUT_IMAGE_SHAPE = (480, 640)
NUMBER_OF_CLASSES = 4

# tensorflow options
BATCH_SIZE = 16
SEED = 1993

# colab setup

## clone repository and setup

In [2]:
if RUNNING_ON_COLAB:    
    # check if repo folder exists, eventually delete it
    %cd '/content/'
    import os
    if os.path.exists('ssd-segmentation'):
        !rm -r 'ssd-segmentation'

    # clone github repository
    !git clone 'https://github.com/matteo-stat/ssd-segmentation.git'

    # copy data from google drive and unzip
    from google.colab import drive
    drive.mount('/content/drive')
    !unzip -o '/content/drive/MyDrive/ssd-segmentation-data/data.zip' -d '/content/ssd-segmentation/'

    # change working directory to cloned repository folder
    %cd '/content/ssd-segmentation/'
    !ls

## run scripts on a venv (example)

```python
# install virtualenv module
!pip install virtualenv

# create a new virtual environment
!virtualenv venv

# check the new virtual environment
!source 'venv/bin/activate'; pip list

# install requirements
!source 'venv/bin/activate'; pip install -r requirements.txt

# run a script
!source '/venv/bin/activate'; python -m 'example-training-multi-target-network.py'
```

# dependecies

In [3]:
import tensorflow as tf
tf.random.set_seed(SEED)
tf.keras.backend.clear_session()

import json
import ssdseglib
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image

# default bounding boxes

In [4]:
# create default bounding boxes
boxes_default = ssdseglib.boxes.DefaultBoundingBoxes(
    feature_maps_shapes=((30, 40), (15, 20), (8, 10), (4, 5)),
    centers_padding_from_borders_percentage=0.025,
    boxes_scales=(0.2, 0.9),
    additional_square_box=True,  
)

# rescale default bounding boxes to input image shape
boxes_default.rescale_boxes_coordinates(image_shape=INPUT_IMAGE_SHAPE)

# input data

## load metadata

In [5]:
# training
with open('data/train.json', 'r') as f:
    path_images_train, path_masks_train, path_labels_boxes_train = map(list, zip(*json.load(f)))

# validation
with open('data/eval.json', 'r') as f:
    path_images_eval, path_masks_eval, path_labels_boxes_eval = map(list, zip(*json.load(f)))

# test
with open('data/test.json', 'r') as f:
    path_images_test, path_masks_test, path_labels_boxes_test = map(list, zip(*json.load(f)))

## data encoder

In [6]:
# create a data reader encoder
data_reader_encoder = ssdseglib.datacoder.DataEncoderDecoder(
    num_classes=NUMBER_OF_CLASSES,
    image_shape=INPUT_IMAGE_SHAPE,
    xmin_boxes_default=boxes_default.get_boxes_coordinates_xmin(coordinates_style='ssd'),
    ymin_boxes_default=boxes_default.get_boxes_coordinates_ymin(coordinates_style='ssd'),
    xmax_boxes_default=boxes_default.get_boxes_coordinates_xmax(coordinates_style='ssd'),
    ymax_boxes_default=boxes_default.get_boxes_coordinates_ymax(coordinates_style='ssd'),
    iou_threshold=0.5,
    standard_deviations_centroids_offsets=(0.1, 0.1, 0.2, 0.2),
    augmentation_horizontal_flip=True
)

## tensorflow datasets

In [7]:
# training
ds_train = (
    tf.data.Dataset.from_tensor_slices((path_images_train, path_masks_train, path_labels_boxes_train))
    .shuffle(buffer_size=len(path_images_train))
    .map(data_reader_encoder.read_and_encode, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size=BATCH_SIZE)
    .map(ssdseglib.datacoder.augmentation_rgb_channels, num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

# validation
ds_eval = (
    tf.data.Dataset.from_tensor_slices((path_images_eval, path_masks_eval, path_labels_boxes_eval))
    .shuffle(buffer_size=len(path_images_eval))
    .map(data_reader_encoder.read_and_encode, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size=BATCH_SIZE)
    .map(ssdseglib.datacoder.augmentation_rgb_channels, num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

# test
ds_test = (
    tf.data.Dataset.from_tensor_slices((path_images_test, path_masks_test, path_labels_boxes_test))
    .shuffle(buffer_size=len(path_images_test))
    .map(data_reader_encoder.read_and_encode, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size=BATCH_SIZE)
    .map(ssdseglib.datacoder.augmentation_rgb_channels, num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

# weighted losses for model training

In [9]:
# weighted loss for semantic segmentation
dice_loss = ssdseglib.losses.dice(classes_weights=(0.1, 0.3, 0.3, 0.3))

# weighted metrics for model training

In [10]:
# weighted metrics for semantic segmentation
jaccard_iou_segmentation_masks_metric = ssdseglib.metrics.jaccard_iou_segmentation_masks(classes_weights=(0.1, 0.3, 0.3, 0.3))

# weighted metrics for boxes classification
categorical_accuracy_metric = ssdseglib.metrics.categorical_accuracy(classes_weights=(0.1, 0.3, 0.3, 0.3))

# weighted metrics for boxes regression
jaccard_iou_bounding_boxes_metric = ssdseglib.metrics.jaccard_iou_bounding_boxes(
    center_x_boxes_default=data_reader_encoder.center_x_boxes_default,
    center_y_boxes_default=data_reader_encoder.center_y_boxes_default,
    width_boxes_default=data_reader_encoder.width_boxes_default,
    height_boxes_default=data_reader_encoder.height_boxes_default,
    standard_deviation_center_x_offsets=data_reader_encoder.standard_deviation_center_x_offsets,
    standard_deviation_center_y_offsets=data_reader_encoder.standard_deviation_center_y_offsets,
    standard_deviation_width_offsets=data_reader_encoder.standard_deviation_width_offsets,
    standard_deviation_height_offsets=data_reader_encoder.standard_deviation_height_offsets
)

# model

## architecture

In [11]:
# mobilenet-v2 with ssdlite and deeplabv3+
model = ssdseglib.models.build_mobilenetv2_ssdseg(
    number_of_boxes_per_point=[
        len(aspect_ratios) + 1 if boxes_default.additional_square_box else 0
        for aspect_ratios in boxes_default.feature_maps_aspect_ratios
    ],
    number_of_classes=NUMBER_OF_CLASSES
)

## optimizer

In [12]:
# put here optimizer and learning rate schedule

## compile

In [15]:
# each ouput has its own loss and metrics
model.compile(
    optimizer='adam',
    loss={
        'output-mask': dice_loss,
        'output-labels': ssdseglib.losses.confidence_loss,
        'output-boxes': ssdseglib.losses.localization_loss
    },
    loss_weights={
        'output-mask': 1.0,
        'output-labels': 1.0,
        'output-boxes': 1.0
    },
    metrics={
        'output-mask': jaccard_iou_segmentation_masks_metric,
        'output-labels': categorical_accuracy_metric,
        'output-boxes': jaccard_iou_bounding_boxes_metric,
    }
)

INFO:tensorflow:Assets written to: model-saved\assets


INFO:tensorflow:Assets written to: model-saved\assets


## fit

In [ ]:
# fit the model
history = model.fit(
    ds_train,
    validation_data=ds_eval,
    epochs=1
)

## plot losses history

In [ ]:
# plot training loss and validation loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

## save weights

In [ ]:
# save model
model.save('model-saved')

In [16]:
# zip and download from colab
if RUNNING_ON_COLAB:
    !zip -r 'model-saved.zip' 'model-saved'
    from google.colab import files
    files.download('model-saved.zip')

## load weights

In [ ]:
# load model
model = tf.keras.models.load_model(
    filepath='model-saved',
    custom_objects={
        'dice_loss': dice_loss,
        'confidence_loss': ssdseglib.losses.confidence_loss,
        'localization_loss': ssdseglib.losses.localization_loss,
        'jaccard_iou_segmentation_masks_metric': jaccard_iou_segmentation_masks_metric,
        'categorical_accuracy_metric': categorical_accuracy_metric,
        'jaccard_iou_bounding_boxes_metric': jaccard_iou_bounding_boxes_metric,
    }
)